In [4]:
%load_ext autoreload
%autoreload 2
from pydarknet import Libdarknet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import ctypes
import os

In [6]:
self = Libdarknet()

In [7]:
self

In [10]:
self.exists

True

In [11]:
self.lib

<CDLL '/mnt/ubuntu1604_2/home/jed/.darknet/libdarknet.so', handle 2079380 at 0x7fea6c908748>

In [12]:
lib = self.lib

In [14]:
# Relative files are **relative** to the darknet directory!
metadata = self.get_metadata("cfg/coco.data")
network = self.load_network("cfg/yolov3.cfg", "weights/yolov3.weights", 0)
img = self.load_image_color("data/dog.jpg")

In [16]:
net_output = self.network_predict_image(network, img)

In [17]:
net_output

In [22]:
num, detections_ptr = self.get_network_boxes(network, img)

In [21]:
class Detections(object):
    def __init__(self, num, detections_ptr):
        self.num=num
        self.detections_ptr=detections_ptr
        
    def __repr__(self):
        return "Detections<{}>".format(self.num)

    

In [23]:
dets = Detections(num, detections_ptr)

In [45]:
for idx in range(num):
    det=detections_ptr[idx]
    print(det.objectness, det.objectness)

0.9996165633201599 0.9996165633201599
0.9953235387802124 0.9953235387802124
0.6891421675682068 0.6891421675682068
0.998834490776062 0.998834490776062
0.7897613644599915 0.7897613644599915
0.9941537976264954 0.9941537976264954
0.9076856970787048 0.9076856970787048
0.8852115869522095 0.8852115869522095
0.7975820302963257 0.7975820302963257
0.6225367784500122 0.6225367784500122
0.6667616963386536 0.6667616963386536
0.5576186180114746 0.5576186180114746
0.5439086556434631 0.5439086556434631


In [30]:
det.classes

80

In [25]:
nms=0.5

In [33]:
self.do_nms_sort(detections_ptr, num, metadata.classes, nms);

In [42]:
detections_ptr